In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('Dataset NLP Fix.csv')
df

,Review,Label
0,Saya merasa terdorong untuk berpartisipasi akt...,1
1,Guru ini selalu memberikan kesempatan kepada s...,1
2,Guru ini ramah dan mudah diajak berdiskusi.,1
3,Sering kali tidak menjelaskan tujuan dari pela...,0
4,Kurang memberikan dukungan kepada siswa yang m...,0
...,...,...
255,Saya merasa guru ini berusaha membuat setiap s...,1
256,Saya merasa guru ini memahami gaya belajar set...,1
257,Sering menggunakan bahasa yang sulit dipahami,0
258,Materi yang diajarkan terlalu banyak teori tan...,0


In [14]:
sentences = df['Review'].to_numpy()
labels = df['Label'].to_numpy()
print(sentences)
print(labels)

['Saya merasa terdorong untuk berpartisipasi aktif dalam pembelajaran.'
 'Guru ini selalu memberikan kesempatan kepada siswa untuk berbicara.'
 'Guru ini ramah dan mudah diajak berdiskusi.'
 'Sering kali tidak menjelaskan tujuan dari pelajaran'
 'Kurang memberikan dukungan kepada siswa yang mengalami kesulitan belajar'
 'Materi yang diajarkan terlalu banyak dalam satu pertemuan'
 'Tidak memberikan kesempatan kepada siswa untuk bertanya'
 'Kurang memberikan dukungan emosional kepada siswa yang mengalami kegagalan'
 'Guru ini membuat suasana kelas menjadi lebih hidup.'
 'Tidak memberikan kesempatan kepada siswa untuk eksplorasi kreativitas'
 'Materi yang diajarkan tidak sesuai dengan kurikulum'
 'Saya merasa guru ini memberikan nilai lebih dalam proses pembelajaran.'
 'Tidak memberikan umpan balik yang mendalam'
 'Tidak memberikan solusi terhadap kesulitan belajar siswa'
 'Materi pelajaran disajikan dengan cara yang mudah dipahami.'
 'Guru ini selalu memberikan umpan balik yang konstrukt

In [15]:
print(sentences.shape)
print(labels.shape)

(260,)
(260,)


In [18]:
#split data
training_sentences = sentences[:200]
training_labels = labels[:200]

validation_sentences = sentences[200:230]
validation_labels = labels[200:230]

testing_sentences = sentences[230:260]
testing_labels = labels[230:260]

In [31]:
vocab_size = 10000
embedding_dim = 16
max_length = 12
trunc_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type)

In [40]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(padded, training_labels, batch_size=128, epochs=100, validation_data=(validation_padded, validation_labels))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 12, 16)            160000    
                                                                 
 bidirectional_7 (Bidirecti  (None, 64)                12544     
 onal)                                                           
                                                                 
 dense_14 (Dense)            (None, 6)                 390       
                                                                 
 dense_15 (Dense)            (None, 1)                 7         
                                                                 
Total params: 172941 (675.55 KB)
Trainable params: 172941 (675.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 5s 939ms/step - loss: 0.6967 

2/2 [==============================] - 0s 40ms/step - loss: 5.4461e-04 - accuracy: 1.0000 - val_loss: 0.0788 - val_accuracy: 0.9667
Epoch 52/100
2/2 [==============================] - 0s 40ms/step - loss: 5.1314e-04 - accuracy: 1.0000 - val_loss: 0.0769 - val_accuracy: 0.9667
Epoch 53/100
2/2 [==============================] - 0s 42ms/step - loss: 4.8544e-04 - accuracy: 1.0000 - val_loss: 0.0749 - val_accuracy: 0.9667
Epoch 54/100
2/2 [==============================] - 0s 39ms/step - loss: 4.6787e-04 - accuracy: 1.0000 - val_loss: 0.0731 - val_accuracy: 0.9667
Epoch 55/100
2/2 [==============================] - 0s 69ms/step - loss: 4.4851e-04 - accuracy: 1.0000 - val_loss: 0.0714 - val_accuracy: 0.9667
Epoch 56/100
2/2 [==============================] - 0s 38ms/step - loss: 4.2873e-04 - accuracy: 1.0000 - val_loss: 0.0698 - val_accuracy: 0.9667
Epoch 57/100
2/2 [==============================] - 0s 37ms/step - loss: 4.1790e-04 - accuracy: 1.0000 - val_loss: 0.0680 - val_accuracy: 0.966

In [42]:
y = model.predict(testing_padded)

1/1 [==============================] - 0s 38ms/step


In [48]:
hasil = []
for data in y:
    if data>0.5:
        hasil.append(1)
    else:
        hasil.append(0)
print(len(hasil))

30


In [50]:
cek = []
for i in range(30):
    if hasil[i] == testing_labels[i]:
        cek.append('Right')
    else:
        cek.append('Wrong')
print(cek)

['Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right']


In [51]:
NLP_SAVED_MODEL = "nlp_saved_model"
tf.saved_model.save(model, NLP_SAVED_MODEL)

INFO:tensorflow:Assets written to: nlp_saved_model\assets


INFO:tensorflow:Assets written to: nlp_saved_model\assets
